## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,80.56,54,0,17.27,clear sky
1,1,Torbay,CA,47.6666,-52.7314,74.17,71,90,25.32,overcast clouds
2,2,Hasaki,JP,35.7333,140.8333,68.40,93,100,14.32,overcast clouds
3,3,Almaznyy,RU,48.0448,40.0450,75.34,84,76,5.91,broken clouds
4,4,Tasiilaq,GL,65.6145,-37.6368,51.96,70,14,5.17,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,80.56,54,0,17.27,clear sky
3,3,Almaznyy,RU,48.0448,40.0450,75.34,84,76,5.91,broken clouds
6,6,Borujerd,IR,33.8973,48.7516,75.85,13,10,4.12,clear sky
7,7,Atuona,PF,-9.8000,-139.0333,77.27,82,19,22.28,light rain
10,10,Gat,IL,31.6100,34.7642,84.40,47,0,9.13,clear sky
14,14,Puerto Ayora,EC,-0.7393,-90.3518,80.56,87,69,4.00,broken clouds
15,15,Trapani,IT,37.8333,12.6667,87.89,43,0,5.75,clear sky
22,22,Xuddur,SO,4.1213,43.8894,78.60,51,100,14.23,overcast clouds
24,24,Marshall,US,34.3834,-86.3333,88.61,66,40,4.00,scattered clouds
36,36,Mana,GF,5.6592,-53.7767,83.17,70,22,7.83,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,80.56,clear sky,-22.9661,-42.0278,
3,Almaznyy,RU,75.34,broken clouds,48.0448,40.0450,
6,Borujerd,IR,75.85,clear sky,33.8973,48.7516,
7,Atuona,PF,77.27,light rain,-9.8000,-139.0333,
10,Gat,IL,84.40,clear sky,31.6100,34.7642,
14,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,
15,Trapani,IT,87.89,clear sky,37.8333,12.6667,
22,Xuddur,SO,78.60,overcast clouds,4.1213,43.8894,
24,Marshall,US,88.61,scattered clouds,34.3834,-86.3333,
36,Mana,GF,83.17,few clouds,5.6592,-53.7767,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"{index} Hotel not found... skipping.")          

3 Hotel not found... skipping.
15 Hotel not found... skipping.
117 Hotel not found... skipping.
127 Hotel not found... skipping.
205 Hotel not found... skipping.
325 Hotel not found... skipping.
364 Hotel not found... skipping.
424 Hotel not found... skipping.
435 Hotel not found... skipping.
477 Hotel not found... skipping.
514 Hotel not found... skipping.
523 Hotel not found... skipping.
530 Hotel not found... skipping.
595 Hotel not found... skipping.
645 Hotel not found... skipping.
648 Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name']!=""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,80.56,clear sky,-22.9661,-42.0278,Pousada Porto Praia
6,Borujerd,IR,75.85,clear sky,33.8973,48.7516,Zagros Hotel
7,Atuona,PF,77.27,light rain,-9.8000,-139.0333,Villa Enata
10,Gat,IL,84.40,clear sky,31.6100,34.7642,OlusHome
14,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [21]:
# 8a. Create the output File (CSV)
output_data_file= 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))